In [57]:
import pandas as pd
import datetime
import requests
import json
import sys, os
import itertools
import time
import numpy as np
from os import path
from pathlib import Path
import heapq

sys.path.insert(0, os.path.join(os.getcwd(), ".."))
import fin_utilities 
cfg = fin_utilities.__cfg_reading()

PROJECT_DIR = eval(cfg['PROJECT_PATH'])
DATA_PATH = PROJECT_DIR / cfg['DATA_FOLDER']
SOURCE = cfg['API_DATA_SOURCE']
URL = cfg[SOURCE]['API_URL_HIST_DATA']
TIMEFRAME = cfg['TIMEFRAME']
COLUMNS = cfg[SOURCE]['COLUMN_NAMES']
PARAMS = cfg[SOURCE]['REQ_PARAMS']
STABLE = cfg['STABLECOIN']

In [128]:
def ts_to_datetime(ts) -> str:
    return datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def request_to_df (
    URL: str,
    PARAMS : dict,
    json_key: str=None
) -> pd.DataFrame:
    req = requests.get(URL, params=PARAMS).json()
    if json_key is not None:
        df = pd.DataFrame(req[json_key])
        return df

    df = pd.DataFrame(req)
    return df

def attach_stable(d_orig, on_key, stable):
    d=d_orig.copy()
    d[on_key] = [value+stable for value in d[on_key]]
    return d


def mask_max_min_pct(percentages):
    """ returns a masked dict with -1 which is max value and 1 for min value
        in order to swap max value against minimum
    """
    max_perc = max(percentages.values())
    min_perc = min(percentages.values())
    idx_max = list(percentages.values()).index(max_perc)
    idx_min = list(percentages.values()).index(min_perc)
    mask =  np.zeros(len(percentages), dtype=int)
    mask[[ idx_min]] = 1
    mask[[idx_max]] = -1
    masked_values = dict(zip(percentages.keys(),mask))

    return masked_values

In [59]:
#get symbols
#req = request_to_df('https://api.binance.com/api/v3/exchangeInfo', json_key='symbols')

In [60]:
def load_data(symbol: str, data_path, URL, PARAMS, col_names, overwrite: bool=False) -> pd.DataFrame:
    
    file_path =  os.path.join(data_path,symbol.replace("/","_").upper() + '_' + TIMEFRAME + '.csv')
    if not path.exists(file_path):
        df = request_to_df(URL,PARAMS)
        df.columns = col_names
        df[['Open','High','Low','Close','Volume','Taker buy base asset volume','Taker buy quote asset volume']] = \
            df[['Open','High','Low','Close','Volume','Taker buy base asset volume','Taker buy quote asset volume']].apply(pd.to_numeric)
        df['OpenTimestamp'] = pd.to_datetime(df['OpenTimestamp'],unit='ms')
        df['CloseTimestamp'] = pd.to_datetime(df['CloseTimestamp'],unit='ms')
        df.drop(columns=df.columns[-1], inplace=True)
        df['symbol'] = symbol
        df.to_csv(file_path, index=False)
    else:
        if overwrite:
            df = request_to_df(URL,PARAMS)
            df.columns = col_names
            df[['Open','High','Low','Close','Volume','Taker buy base asset volume','Taker buy quote asset volume']] = \
                df[['Open','High','Low','Close','Volume','Taker buy base asset volume','Taker buy quote asset volume']].apply(pd.to_numeric)
            df['OpenTimestamp'] = pd.to_datetime(df['OpenTimestamp'],unit='ms')
            df['CloseTimestamp'] = pd.to_datetime(df['CloseTimestamp'],unit='ms')
            df.drop(columns=df.columns[-1], inplace=True)
            df.to_csv(file_path, index=False)
        else:
            df = pd.read_csv(file_path)
            df['symbol'] = symbol

    return df

In [61]:
LIST_PARAMS = attach_stable(PARAMS, 'symbol', STABLE)
LIST_PARAMS  = [dict(zip(LIST_PARAMS.keys(), el)) for el in itertools.product(*LIST_PARAMS.values())]

In [62]:
results = pd.DataFrame()
for param in LIST_PARAMS:
    tmp = load_data(str.replace(param['symbol'],STABLE,''), DATA_PATH, URL, param, COLUMNS)
    results = pd.concat([results, tmp], axis=0).reset_index(drop=True)

keep_cols = ['OpenTimestamp', 'Open', 'Close', 'symbol']
results = results[keep_cols]
results['pct_change'] = np.round((results['Close']-results['Open'])/results['Open'],5)

In [63]:
results

,OpenTimestamp,Open,Close,symbol,pct_change
0,2020-01-07,4.111,4.075,ATOM,-0.00876
1,2020-01-08,4.087,3.995,ATOM,-0.02251
2,2020-01-09,3.995,4.245,ATOM,0.06258
3,2020-01-10,4.244,4.270,ATOM,0.00613
4,2020-01-11,4.273,4.264,ATOM,-0.00211
...,...,...,...,...,...
2995,2022-09-28,19078.100,19412.820,BTC,0.01754
2996,2022-09-29,19412.820,19591.510,BTC,0.00920
2997,2022-09-30,19590.540,19422.610,BTC,-0.00857
2998,2022-10-01,19422.610,19310.950,BTC,-0.00575


In [64]:
params_combination = [el for el in itertools.combinations(PARAMS['symbol'],2)]
print(len(params_combination))
params_comb= []
for el in range(len(params_combination)):
    #params_comb.append(''.join(el))
    params_combination[el] = list(params_combination[el])

params_combination

3


[['ATOM', 'ETH'], ['ATOM', 'BTC'], ['ETH', 'BTC']]

In [65]:
pivot_results = pd.DataFrame()
key_join = 'OpenTimestamp'
for pair in results['symbol'].unique():
    if list(results['symbol'].unique()).index(pair)==0:
        pivot_results = results[results['symbol']==pair]
        pivot_results.columns = [pair+'_'+col if col != key_join  else col for col in pivot_results.columns]
    else:
        tmp = results[results['symbol']==pair]
        tmp.columns = [pair+'_'+col if col != key_join  else col for col in tmp.columns]
        pivot_results = pivot_results.merge(tmp, on=key_join)

pivot_results.drop(columns=[c for c in pivot_results.columns if 'symbol' in c], inplace=True)

initial_swap_price = list(pivot_results[[col for col  in pivot_results.columns if 'Close' in col]].iloc[0]) + [1]
#initial_swap_price

single_pct_cols = pivot_results[[col for col  in pivot_results.columns if 'pct_change' in col]]
close_prices = pivot_results[[col for col  in pivot_results.columns if 'Close' in col]]

In [66]:
for comb in params_combination:
    print(comb)
    pivot_results[comb[0]+comb[1]+'_Open'] = pivot_results[comb[0]+'_Open']/pivot_results[comb[1]+'_Open']
    pivot_results[comb[0]+comb[1]+'_Close'] = pivot_results[comb[0]+'_Close']/pivot_results[comb[1]+'_Close']
    pivot_results[comb[0]+comb[1]+'_pct_change'] = np.round((pivot_results[comb[0]+comb[1]+'_Close']-pivot_results[comb[0]+comb[1]+'_Open'])/pivot_results[comb[0]+comb[1]+'_Open'],5)

pivot_results

['ATOM', 'ETH']
['ATOM', 'BTC']
['ETH', 'BTC']


,OpenTimestamp,ATOM_Open,ATOM_Close,ATOM_pct_change,ETH_Open,ETH_Close,ETH_pct_change,BTC_Open,BTC_Close,BTC_pct_change,ATOMETH_Open,ATOMETH_Close,ATOMETH_pct_change,ATOMBTC_Open,ATOMBTC_Close,ATOMBTC_pct_change,ETHBTC_Open,ETHBTC_Close,ETHBTC_pct_change
0,2020-01-07,4.111,4.075,-0.00876,144.14,142.80,-0.00930,7758.90,8145.28,0.04980,0.028521,0.028536,0.00054,0.000530,0.000500,-0.05578,0.018577,0.017532,-0.05629
1,2020-01-08,4.087,3.995,-0.02251,142.80,140.72,-0.01457,8145.92,8055.98,-0.01104,0.028620,0.028390,-0.00806,0.000502,0.000496,-0.01160,0.017530,0.017468,-0.00356
2,2020-01-09,3.995,4.245,0.06258,140.76,137.74,-0.02145,8054.72,7817.76,-0.02942,0.028382,0.030819,0.08588,0.000496,0.000543,0.09479,0.017475,0.017619,0.00821
3,2020-01-10,4.244,4.270,0.00613,137.73,144.84,0.05162,7817.74,8197.02,0.04852,0.030814,0.029481,-0.04326,0.000543,0.000521,-0.04043,0.017618,0.017670,0.00296
4,2020-01-11,4.273,4.264,-0.00211,144.83,142.38,-0.01692,8198.86,8020.01,-0.02181,0.029504,0.029948,0.01506,0.000521,0.000532,0.02015,0.017665,0.017753,0.00501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2022-09-28,13.489,12.997,-0.03647,1328.01,1337.20,0.00692,19078.10,19412.82,0.01754,0.010157,0.009720,-0.04310,0.000707,0.000670,-0.05309,0.069609,0.068882,-0.01044
996,2022-09-29,12.997,12.771,-0.01739,1337.20,1335.70,-0.00112,19412.82,19591.51,0.00920,0.009720,0.009561,-0.01629,0.000670,0.000652,-0.02635,0.068882,0.068177,-0.01023
997,2022-09-30,12.773,13.024,0.01965,1335.69,1328.72,-0.00522,19590.54,19422.61,-0.00857,0.009563,0.009802,0.02500,0.000652,0.000671,0.02847,0.068180,0.068411,0.00338
998,2022-10-01,13.023,12.682,-0.02618,1328.71,1311.79,-0.01273,19422.61,19310.95,-0.00575,0.009801,0.009668,-0.01362,0.000671,0.000657,-0.02055,0.068410,0.067930,-0.00703


In [67]:
tokens_list = PARAMS['symbol']+[STABLE]
initial_budget = 1200
percentages = [1/4,1/4,1/4,1/4]
equity = [el*initial_budget for el in percentages]
portfolio =  dict(zip(tokens_list, equity))
#crypto_equivalent = [a*b for a,b in zip(lista,listb)]
portfolio_performance = pd.DataFrame(columns=['Timestamp','ATOM','ETH','BTC','USDT', 'ATOM_value', 'ETH_value', 'BTC_value', 'USDT_value', 'ATOM_pct','ETH_pct','BTC_pct','USDT_pct', 'Total_Value', 'Action'])
#portfolio[['BTC','ATOM','USDT']].loc[0] = 
portfolio_performance

,Timestamp,ATOM,ETH,BTC,USDT,ATOM_value,ETH_value,BTC_value,USDT_value,ATOM_pct,ETH_pct,BTC_pct,USDT_pct,Total_Value,Action


In [68]:
pivot_results.head(5)

,OpenTimestamp,ATOM_Open,ATOM_Close,ATOM_pct_change,ETH_Open,ETH_Close,ETH_pct_change,BTC_Open,BTC_Close,BTC_pct_change,ATOMETH_Open,ATOMETH_Close,ATOMETH_pct_change,ATOMBTC_Open,ATOMBTC_Close,ATOMBTC_pct_change,ETHBTC_Open,ETHBTC_Close,ETHBTC_pct_change
0,2020-01-07,4.111,4.075,-0.00876,144.14,142.80,-0.00930,7758.90,8145.28,0.04980,0.028521,0.028536,0.00054,0.000530,0.000500,-0.05578,0.018577,0.017532,-0.05629
1,2020-01-08,4.087,3.995,-0.02251,142.80,140.72,-0.01457,8145.92,8055.98,-0.01104,0.028620,0.028390,-0.00806,0.000502,0.000496,-0.01160,0.017530,0.017468,-0.00356
2,2020-01-09,3.995,4.245,0.06258,140.76,137.74,-0.02145,8054.72,7817.76,-0.02942,0.028382,0.030819,0.08588,0.000496,0.000543,0.09479,0.017475,0.017619,0.00821
3,2020-01-10,4.244,4.270,0.00613,137.73,144.84,0.05162,7817.74,8197.02,0.04852,0.030814,0.029481,-0.04326,0.000543,0.000521,-0.04043,0.017618,0.017670,0.00296
4,2020-01-11,4.273,4.264,-0.00211,144.83,142.38,-0.01692,8198.86,8020.01,-0.02181,0.029504,0.029948,0.01506,0.000521,0.000532,0.02015,0.017665,0.017753,0.00501


In [69]:
pair_pct_cols = pivot_results[[col for col  in pivot_results.columns if 'pct_change' in col and col not in single_pct_cols.columns]]
pair_pct_cols.head(5)

,ATOMETH_pct_change,ATOMBTC_pct_change,ETHBTC_pct_change
0,0.00054,-0.05578,-0.05629
1,-0.00806,-0.01160,-0.00356
2,0.08588,0.09479,0.00821
3,-0.04326,-0.04043,0.00296
4,0.01506,0.02015,0.00501


In [70]:
single_pct_cols.head(5)

,ATOM_pct_change,ETH_pct_change,BTC_pct_change
0,-0.00876,-0.00930,0.04980
1,-0.02251,-0.01457,-0.01104
2,0.06258,-0.02145,-0.02942
3,0.00613,0.05162,0.04852
4,-0.00211,-0.01692,-0.02181


In [153]:
tmp = [eq/val for eq,val in zip(equity,initial_swap_price)]
crypto_amount = dict(zip(tokens_list,tmp))
print("Initial Amount: {}".format(crypto_amount))
tmp = [eq*val for eq,val in zip(crypto_amount.values(),initial_swap_price)]
crypto_values = dict(zip(tokens_list,tmp))
print("Initial Values: {}".format(crypto_val))
crypto_percentages = dict(zip(tokens_list, percentages))
print("Initial Pct: {}".format(crypto_percentages))

Initial Amount: {'ATOM': 73.61963190184049, 'ETH': 2.1008403361344534, 'BTC': 0.03683114638170818, 'USDT': 300.0}
Initial Values: {'ATOM': 300.0, 'ETH': 300.0, 'BTC': 300.0, 'USDT': 300.0}
Initial Pct: {'ATOM': 0.25, 'ETH': 0.25, 'BTC': 0.25, 'USDT': 0.25}


In [142]:
def swap_tokens(token_list,swap_values, variation_value, variation_pct, new_crypto_val, crypto_amount, current_close_price, fees):
    
    print("___________________________________________START SWAPPING___________________________________________")
    print(f'Swap Values {swap_values}')
    print(variation_value)
    amount_to_swap = (max(np.abs(list(variation_value.values())))+min(np.abs(list(variation_value.values()))))/2
    #amount_to_swap = max(variation_value.values())/2
    print('Amount to Swap:{}'.format(amount_to_swap))
    amount_to_swap = np.round(amount_to_swap*(1-fees),3)
    print('Amount to Swap net of taxes:{}'.format(amount_to_swap))
    swapping = [var*flg for var,flg in zip([amount_to_swap]*len(swap_values),list(swap_values.values()))]
    dollar_swap_values = np.asarray(list(swap_values.values()))*amount_to_swap
    swap_amount = dict(zip(token_list,[np.round(val/price,5) for val,price in zip(list(dollar_swap_values), list(current_close_price.values()))]))
    swap_amount_order_only = {k:v for k,v in swap_amount.items() if v != 0}
    _, v = next(iter(swap_amount_order_only.items()))
    #print(swap_amount_order_only)
    abs_swap_amount = dict(zip(token_list,np.abs(list(swap_amount.values()))))
    swap_amount_print_only = {k:v for k,v in abs_swap_amount.items() if v != 0}
    #print(swap_amount_print_only)
    if v < 0:
        print(f'Swapping {swap_amount_print_only[list(swap_amount_print_only.keys())[0]]} {list(swap_amount_print_only.keys())[0]} for {swap_amount_print_only[list(swap_amount_print_only.keys())[1]]} {list(swap_amount_print_only.keys())[1]}')
    else:
        print(f'Swapping {swap_amount_print_only[list(swap_amount_print_only.keys())[1]]} {list(swap_amount_print_only.keys())[1]} for {swap_amount_print_only[list(swap_amount_print_only.keys())[0]]} {list(swap_amount_print_only.keys())[0]}')
    new_crypto_amount = dict(zip(token_list,[old+new for old,new in zip(list(crypto_amount.values()),list(swap_amount.values()))]))
    print("New Amount: {}".format(new_crypto_amount))
    new_crypto_values =  dict(zip(token_list,[amt*val for amt,val in zip(list(new_crypto_amount.values()),list(current_close_price.values()))]))
    print("New Values: {}".format(new_crypto_values))
    tmp_sum = sum(list(new_crypto_values.values()))
    tmp = np.round([i/tmp_sum for i in list(new_crypto_values.values())],5)
    new_crypto_percentages = dict(zip(tokens_list,tmp))
    print("New Percentages: {}".format(new_crypto_percentages))
    print('Portfolio value: {}'.format(tmp_sum))

    return new_crypto_amount, new_crypto_percentages, new_crypto_values

def update_only(token_list,swap_values, variation_value, variation_pct, new_crypto_val, crypto_amount, current_close_price, fees):
    
    print("___________________________________________ UPDATE ________________________________________________")
    print("New Amount: {}".format(crypto_amount))
    new_crypto_values =  dict(zip(token_list,[amt*val for amt,val in zip(list(crypto_amount.values()),list(current_close_price.values()))]))
    print("New Values: {}".format(new_crypto_values))
    tmp_sum = sum(tmp)
    print("New Sum: {}".format(tmp_sum))


In [ ]:
######################################################################## SWAP BASATO SU QUELLA CHE PERFORMA MEGLIO VS QUELLA CHE PERFORMA PEGGIO ############################################
# IL PROBLEMA è CHE NON VIENE MANTENUTA LA PROPORZIONE DEL PORTAFOGLIO  
# PROVO LA STRADA DI SWAPPARE IN BASE AL PORTAFOGLIO E BASTA

fees=0.01
print("Fees: {}%".format(100*fees))

for i in range(len(pivot_results[pivot_results.columns])):
    print ("********************************************************************* ITERATION {} *********************************************************************".format(i))
    if i==0: continue
    tmp = [eq*(1+val) for eq,val in zip(crypto_values.values(),list(single_pct_cols.loc[i]) + [0])]
    print(tmp)
    current_close_price = dict(zip(tokens_list,list(close_prices.loc[i]) + [1.0]))
    print("Current Close: {}".format(current_close_price))
    new_crypto_val = dict(zip(tokens_list,tmp))
    #print("New Values: {}".format(new_crypto_val))
    tmp_sum = sum(tmp)
    #print("New Sum: {}".format(tmp_sum))
    tmp = np.round([i/tmp_sum for i in tmp],5)
    new_crypto_pct = dict(zip(tokens_list,tmp))
    #print("New Normalized Percentages: {}".format(new_crypto_pct))
    tmp = np.round([new-old for new,old in zip(new_crypto_pct.values(),crypto_percentages.values())],5)
    norm_delta_pct = dict(zip(tokens_list, tmp))
    #print("New Delta Normalized Percentages: {}".format(norm_delta_pct))
    tmp = np.round([(current-init)/init for init,current in zip(crypto_values.values(),new_crypto_val.values())],5)    
    variation_pct = dict(zip(tokens_list, tmp))
    #print("Variation Percentage: {}".format(variation_pct))
    swap_values = mask_max_min_pct(norm_delta_pct)
    tmp = [val*pct for val,pct in zip (crypto_values.values(), variation_pct.values())]
    variation_value = dict(zip(tokens_list, tmp))
    #print("Variation Value: {}".format(variation_value))
    tmp = [val*pct for val,pct in zip (crypto_values.values(), variation_pct.values())]    
    #print("Best (-1) & Worst (1): {}".format(swap_values))
    #swap_tokens()
    crypto_amount, crypto_percentages, crypto_values = swap_tokens(tokens_list, swap_values, variation_value, norm_delta_pct, new_crypto_val, crypto_amount, current_close_price, fees)

In [ ]:
######################################################################## SWAP BASATO SULLE PERCENTUALI #########################################################################

fees=0.01
print("Fees: {}%".format(100*fees))

for i in range(len(pivot_results[pivot_results.columns])):
    print ("********************************************************************* ITERATION {} *********************************************************************".format(i))
    if i==0: continue
    tmp = [eq*(1+val) for eq,val in zip(crypto_values.values(),list(single_pct_cols.loc[i]) + [0])]
    print(tmp)
    current_close_price = dict(zip(tokens_list,list(close_prices.loc[i]) + [1.0]))
    print("Current Close: {}".format(current_close_price))
    new_crypto_val = dict(zip(tokens_list,tmp))
    #print("New Values: {}".format(new_crypto_val))
    tmp_sum = sum(tmp)
    #print("New Sum: {}".format(tmp_sum))
    tmp = np.round([i/tmp_sum for i in tmp],5)
    new_crypto_pct = dict(zip(tokens_list,tmp))
    #print("New Normalized Percentages: {}".format(new_crypto_pct))
    tmp = np.round([new-old for new,old in zip(new_crypto_pct.values(),crypto_percentages.values())],5)
    norm_delta_pct = dict(zip(tokens_list, tmp))
    #print("New Delta Normalized Percentages: {}".format(norm_delta_pct))
    tmp = np.round([(current-init)/init for init,current in zip(crypto_values.values(),new_crypto_val.values())],5)    
    variation_pct = dict(zip(tokens_list, tmp))
    #print("Variation Percentage: {}".format(variation_pct))
    swap_values = mask_max_min_pct(new_crypto_pct)
    tmp = [val*pct for val,pct in zip (crypto_values.values(), variation_pct.values())]
    variation_value = dict(zip(tokens_list, tmp))
    #print("Variation Value: {}".format(variation_value))
    tmp = [val*pct for val,pct in zip (crypto_values.values(), variation_pct.values())]    
    #print("Best (-1) & Worst (1): {}".format(swap_values))
    #swap_tokens()
    
    crypto_amount, crypto_percentages, crypto_values = swap_tokens(tokens_list, swap_values, variation_value, norm_delta_pct, new_crypto_val, crypto_amount, current_close_price, fees)

In [152]:
######################################################################## SWAP BASATO SULLE PERCENTUALI #########################################################################

fees=0.01
print("Fees: {}%".format(100*fees))

for i in range(len(pivot_results[pivot_results.columns])):
    print ("********************************************************************* ITERATION {} *********************************************************************".format(i))
    if i==0: continue
    tmp = [eq*(1+val) for eq,val in zip(crypto_values.values(),list(single_pct_cols.loc[i]) + [0])]
    print(tmp)
    print("Current Amount: {}".format(crypto_amount))
    current_close_price = dict(zip(tokens_list,list(close_prices.loc[i]) + [1.0]))
    print("Current Close: {}".format(current_close_price))
    new_crypto_val = dict(zip(tokens_list,tmp))
    print("New Values: {}".format(new_crypto_val))
    tmp_sum = sum(tmp)
    print("New Sum: {}".format(tmp_sum))
    tmp = np.round([i/tmp_sum for i in tmp],5)
    new_crypto_pct = dict(zip(tokens_list,tmp))
    #print("New Normalized Percentages: {}".format(new_crypto_pct))
    tmp = np.round([new-old for new,old in zip(new_crypto_pct.values(),crypto_percentages.values())],5)
    norm_delta_pct = dict(zip(tokens_list, tmp))
    #print("New Delta Normalized Percentages: {}".format(norm_delta_pct))
    tmp = np.round([(current-init)/init for init,current in zip(crypto_values.values(),new_crypto_val.values())],5)    
    variation_pct = dict(zip(tokens_list, tmp))
    #print("Variation Percentage: {}".format(variation_pct))
    tmp = [val*pct for val,pct in zip (crypto_values.values(), variation_pct.values())]
    variation_value = dict(zip(tokens_list, tmp))
    #print("Variation Value: {}".format(variation_value))
    tmp = [val*pct for val,pct in zip (crypto_values.values(), variation_pct.values())]    
    #print("Best (-1) & Worst (1): {}".format(swap_values))
    #swap_tokens()
    print('PERCENTUALI!!!!')
    print(new_crypto_pct)
    print(norm_delta_pct)
    swap_values = mask_max_min_pct(new_crypto_pct)
    swap_values_2 = mask_max_min_pct(norm_delta_pct)
    if swap_values == swap_values_2:
        crypto_amount, crypto_percentages, crypto_values = swap_tokens(tokens_list, swap_values, variation_value, norm_delta_pct, new_crypto_val, crypto_amount, current_close_price, fees)
    else:
        print('nothing done')

Fees: 1.0%
********************************************************************* ITERATION 0 *********************************************************************
********************************************************************* ITERATION 1 *********************************************************************
[293.247, 295.629, 296.688, 300.0]
Current Amount: {'ATOM': 73.61963190184049, 'ETH': 2.1008403361344534, 'BTC': 0.03683114638170818, 'USDT': 300.0}
Current Close: {'ATOM': 3.995, 'ETH': 140.72, 'BTC': 8055.98, 'USDT': 1.0}
New Values: {'ATOM': 293.247, 'ETH': 295.629, 'BTC': 296.688, 'USDT': 300.0}
New Sum: 1185.5639999999999
PERCENTUALI!!!!
{'ATOM': 0.24735, 'ETH': 0.24936, 'BTC': 0.25025, 'USDT': 0.25304}
{'ATOM': -0.00265, 'ETH': -0.00064, 'BTC': 0.00025, 'USDT': 0.00304}
___________________________________________START SWAPPING___________________________________________
Swap Values {'ATOM': 1, 'ETH': 0, 'BTC': 0, 'USDT': -1}
{'ATOM': -6.752999999999999, 'ETH': -4.37099999

In [52]:
pivot_results[pivot_results.columns]

,OpenTimestamp,ATOM_Open,ATOM_Close,ATOM_pct_change,ETH_Open,ETH_Close,ETH_pct_change,BTC_Open,BTC_Close,BTC_pct_change,ATOMETH_Open,ATOMETH_Close,ATOMETH_pct_change,ATOMBTC_Open,ATOMBTC_Close,ATOMBTC_pct_change,ETHBTC_Open,ETHBTC_Close,ETHBTC_pct_change
0,2020-01-07,4.111,4.075,-0.00876,144.14,142.80,-0.00930,7758.90,8145.28,0.04980,0.028521,0.028536,0.00054,0.000530,0.000500,-0.05578,0.018577,0.017532,-0.05629
1,2020-01-08,4.087,3.995,-0.02251,142.80,140.72,-0.01457,8145.92,8055.98,-0.01104,0.028620,0.028390,-0.00806,0.000502,0.000496,-0.01160,0.017530,0.017468,-0.00356
2,2020-01-09,3.995,4.245,0.06258,140.76,137.74,-0.02145,8054.72,7817.76,-0.02942,0.028382,0.030819,0.08588,0.000496,0.000543,0.09479,0.017475,0.017619,0.00821
3,2020-01-10,4.244,4.270,0.00613,137.73,144.84,0.05162,7817.74,8197.02,0.04852,0.030814,0.029481,-0.04326,0.000543,0.000521,-0.04043,0.017618,0.017670,0.00296
4,2020-01-11,4.273,4.264,-0.00211,144.83,142.38,-0.01692,8198.86,8020.01,-0.02181,0.029504,0.029948,0.01506,0.000521,0.000532,0.02015,0.017665,0.017753,0.00501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2022-09-28,13.489,12.997,-0.03647,1328.01,1337.20,0.00692,19078.10,19412.82,0.01754,0.010157,0.009720,-0.04310,0.000707,0.000670,-0.05309,0.069609,0.068882,-0.01044
996,2022-09-29,12.997,12.771,-0.01739,1337.20,1335.70,-0.00112,19412.82,19591.51,0.00920,0.009720,0.009561,-0.01629,0.000670,0.000652,-0.02635,0.068882,0.068177,-0.01023
997,2022-09-30,12.773,13.024,0.01965,1335.69,1328.72,-0.00522,19590.54,19422.61,-0.00857,0.009563,0.009802,0.02500,0.000652,0.000671,0.02847,0.068180,0.068411,0.00338
998,2022-10-01,13.023,12.682,-0.02618,1328.71,1311.79,-0.01273,19422.61,19310.95,-0.00575,0.009801,0.009668,-0.01362,0.000671,0.000657,-0.02055,0.068410,0.067930,-0.00703


In [ ]:
z = dict({'eth': 0.05, 'btv':1})

In [ ]:
input_list = [1,2,3,4,5]
number_of_elements = 3
heapq.nlargest(2, input_list)